# Shelter Animal Outcomes 5

## Naive Bayes

In [1]:
from sklearn.naive_bayes import GaussianNB 
from sklearn import cross_validation
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from time import time
from operator import itemgetter
import numpy as np
import pandas as pd

In [2]:
df_train = pd.read_csv('../Shelter_train.csv')
df_test = pd.read_csv('../Shelter_test.csv')

In [3]:
X = df_train.ix[:, :-1]
y = df_train.ix[:, -1]
df_test = df_test.drop('ID', 1)

In [4]:
clf = GaussianNB()
cross_validation.cross_val_score(clf, X, y, scoring="log_loss")

array([-1.47093332, -1.58296243, -1.46664248])

In [5]:
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

In [6]:
params = {
    "featureSelection__k" : [2, 3, 4, 5, 6, 7, 8]
    }

In [7]:
pipeline = Pipeline([
        ('featureSelection', SelectKBest(f_classif)),
        ('clf', GaussianNB())
    ])
grid_search = GridSearchCV(pipeline, params, n_jobs=-1, scoring='log_loss')
start = time()
grid_search.fit(X, y)
print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.grid_scores_)))
report(grid_search.grid_scores_)
predictions = grid_search.predict_proba(df_test)
output = pd.DataFrame(predictions, columns=['Adoption', 'Died', 'Euthanasia', 'Return_to_owner', 'Transfer'])
output.index.names = ['ID']
output.index += 1
output.head()

GridSearchCV took 0.59 seconds for 7 candidate parameter settings.
Model with rank: 1
Mean validation score: -1.068 (std: 0.004)
Parameters: {'featureSelection__k': 2}

Model with rank: 2
Mean validation score: -1.075 (std: 0.008)
Parameters: {'featureSelection__k': 3}

Model with rank: 3
Mean validation score: -1.118 (std: 0.010)
Parameters: {'featureSelection__k': 4}



,Adoption,Died,Euthanasia,Return_to_owner,Transfer
1,0.002450,0.016726,0.115034,0.093750,0.772040
2,0.622077,0.001029,0.017173,0.249382,0.110339
3,0.646979,0.001344,0.026792,0.175275,0.149610
4,0.129043,0.010029,0.082384,0.293138,0.485407
5,0.647427,0.000379,0.010397,0.276589,0.065208


In [8]:
output.to_csv('../submission-GaussianNB.3.0.csv', index_label = 'ID')